<a href="https://colab.research.google.com/github/RichardDominik/neural-networks-CV/blob/master/NN_CV_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>